In [35]:
# things we need for NLP
import nltk
#nltk.download('punkt')

In [36]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [37]:
# things we need for Tensorflow
import numpy as np

In [38]:
import tflearn
import tensorflow as tf
import random

In [39]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [43]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))
print('documents:',documents)
print('------')
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

documents: [(['Hello'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'), (['What', 'are', 'your', 'hours', '?'], 'hours'), (['When', 'are', 'you', 'open', '?'], 'hours'), (['Which', 'mopeds', 'do', 'you', 'have', '?'], 'mopeds'), (['What', 'kinds', 'of', 'mopeds', 'are', 'there', '?'], 'mopeds'), (['What', 'do', 'you', 'rent', '?'], 'mopeds'), (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'), (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'), (['Are', 'you', 'cash', 'only', '?'], 'payments'), (['Are', 'you', 'open', 'today', '?'], 'opentoday'), (['When', 'do', 'you', 'open', 'today', '?'], 'opentoday'), (['What', 'are', 'your', 'hours'

In [46]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    print(pattern_words)
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])


['hello']
['how', 'ar', 'you']
['is', 'anyon', 'ther', '?']
['hello']
['good', 'day']
['bye']
['see', 'you', 'lat']
['goodby']
['thank']
['thank', 'you']
['that', "'s", 'help']
['what', 'hour', 'ar', 'you', 'op', '?']
['what', 'ar', 'yo', 'hour', '?']
['when', 'ar', 'you', 'op', '?']
['which', 'mop', 'do', 'you', 'hav', '?']
['what', 'kind', 'of', 'mop', 'ar', 'ther', '?']
['what', 'do', 'you', 'rent', '?']
['do', 'you', 'tak', 'credit', 'card', '?']
['do', 'you', 'acceiv', 'mastercard', '?']
['ar', 'you', 'cash', 'on', '?']
['ar', 'you', 'op', 'today', '?']
['when', 'do', 'you', 'op', 'today', '?']
['what', 'ar', 'yo', 'hour', 'today', '?']
['can', 'we', 'rent', 'a', 'mop', '?']
['i', "'d", 'lik', 'to', 'rent', 'a', 'mop']
['how', 'doe', 'thi', 'work', '?']
['today']


In [47]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=2000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 7999  | total loss: 0.07322 | time: 0.027s
| Adam | epoch: 2000 | loss: 0.07322 - acc: 0.9971 -- iter: 24/27
Training Step: 8000  | total loss: 0.07122 | time: 0.037s
| Adam | epoch: 2000 | loss: 0.07122 - acc: 0.9974 -- iter: 27/27
--
INFO:tensorflow:/home/mqm/Workspace/dl/notebooks/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [48]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [49]:
p = bow("Any time", words,True)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]
['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']


In [50]:
print(model.predict([p]))

[[ 0.036187    0.36783069  0.00776291  0.01266936  0.00224014  0.32677537
   0.02668541  0.00131436  0.21853468]]


In [51]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )